In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Transformations


# Dataset: folder structure
# dataset/
#    charts/
#    non_charts/
dataset = datasets.ImageFolder(root='dataset', transform=transform)

# DataLoader
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# Check classes
print(dataset.classes)  # ['charts', 'non_charts']

# Example iteration
for imgs, labels in dataloader:
    print(imgs.shape, labels)
    break


In [1]:
import torch

In [16]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [17]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor()
])

In [18]:
dataset = datasets.ImageFolder(root='dataset', transform=transform)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

for imgs, labels in dataloader:
    print(imgs.shape, labels)
    break


torch.Size([4, 3, 224, 224]) tensor([0, 1, 1, 0])


In [19]:
import torch.nn as nn
import torch.optim as optim

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

transform = transforms.Compose([
    transforms.RandomResizedCrop(128), 
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    transforms.ToTensor()
])

dataset = datasets.ImageFolder(root='dataset', transform=transform)

train_size = int(0.75 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=2)

class TinyCNN(nn.Module):
    def __init__(self):
        super(TinyCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 8, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(8, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(16*32*32, 32),
            nn.ReLU(),
            nn.Linear(32, 2)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x

model = TinyCNN()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# -----------------
# 4. Training Loop
# -----------------
epochs = 20
for epoch in range(epochs):
    model.train()
    for imgs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    # Validation
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            outputs = model(imgs)
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    acc = correct / total
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}, Val Acc: {acc:.2f}")


Epoch 1/20, Loss: 0.9297, Val Acc: 0.00
Epoch 2/20, Loss: 0.5768, Val Acc: 0.50
Epoch 3/20, Loss: 0.8639, Val Acc: 0.50
Epoch 4/20, Loss: 0.8277, Val Acc: 0.50
Epoch 5/20, Loss: 0.6110, Val Acc: 0.50
Epoch 6/20, Loss: 0.6160, Val Acc: 0.50
Epoch 7/20, Loss: 0.8512, Val Acc: 0.50
Epoch 8/20, Loss: 0.8555, Val Acc: 0.50
Epoch 9/20, Loss: 0.6001, Val Acc: 0.50
Epoch 10/20, Loss: 0.8351, Val Acc: 0.50
Epoch 11/20, Loss: 0.5816, Val Acc: 0.50
Epoch 12/20, Loss: 0.8153, Val Acc: 0.50
Epoch 13/20, Loss: 0.6049, Val Acc: 0.50
Epoch 14/20, Loss: 0.5788, Val Acc: 0.50
Epoch 15/20, Loss: 0.5648, Val Acc: 0.50
Epoch 16/20, Loss: 0.8503, Val Acc: 0.50
Epoch 17/20, Loss: 0.5624, Val Acc: 0.50
Epoch 18/20, Loss: 0.5645, Val Acc: 0.50
Epoch 19/20, Loss: 0.9303, Val Acc: 0.50
Epoch 20/20, Loss: 0.4857, Val Acc: 0.50


In [44]:
torch.save(model.state_dict(), "tiny_chart_cnn.pth")
print("Model saved as tiny_chart_cnn.pth")

Model saved as tiny_chart_cnn.pth


In [52]:
from PIL import Image
import torch
from torchvision import transforms

test_img = Image.open("page.png").convert("RGB")

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])
input_tensor = transform(test_img).unsqueeze(0)

model = TinyCNN()
model.load_state_dict(torch.load("tiny_chart_cnn.pth"))
model.eval()

with torch.no_grad():
    output = model(input_tensor)
    pred = torch.argmax(output, dim=1).item()

if pred == 0:
    print("This image is a CHART")
else:
    print("This image is NON-CHART")


This image is NON-CHART
